## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf
#tf.enable_eager_execution()

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


In [2]:
print(tf.__version__)

1.15.0


In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


### Collect Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

In [0]:
#dataset = np.loadtxt("/content/drive/My Drive/ANN Mahesh Anand/pima-indians-diabetes.data", delimiter=",")
data = pd.read_csv('/content/drive/My Drive/Internal Lab-6/prices.csv')

### Check all columns in the dataset

In [5]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

In [16]:
data.shape

(851264, 7)

In [17]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(["date", "symbol"], axis=1)

In [19]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data = data.iloc[:1000,:]

In [21]:
data.shape

(1000, 5)

In [0]:
data["volume"] = data["volume"]/1000000

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


In [0]:
X = data.drop("volume", axis=1)

In [0]:
y = data["volume"]

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.30,random_state = 7)

In [15]:
X_train.shape

(700, 4)

#### Convert Training and Test Data to numpy float32 arrays


In [17]:
X_train.dtypes
y_train.dtypes

dtype('float64')

In [0]:
X_train =np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')
y_train =np.array(y_train).astype('float32')
y_test = np.array(y_test).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [18]:
y_train.shape

(700,)

In [0]:
y_train = y_train.reshape(-1,1)

In [0]:
#We are initializing weights and Bias of (Input-Hidden Layer)
w1 = tf.random_normal(shape=(4,5))
b1 = tf.zeros(shape=(5))

In [0]:
#We are initializing weights and Bias of (Hidden-Output)
w2 = tf.random_normal(shape=(5,1))
b2 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w1, b1,w2,b2):
    
    xw_matmul = tf.matmul(x, w1)
    net1 = tf.add(xw_matmul, b1)
    y=tf.sigmoid(net1)
    net2=tf.matmul(y, w2)+b2
    out=tf.sigmoid(net2)
    return net2,out

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(train_x, train_y, w1, b1,w2,b2, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w1,b1,w2,b2])
        
        net2,current_prediction = prediction(train_x, w1, b1,w2,b2)
        current_loss =loss(net2,train_y)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1,w2,b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1
    
    return w1, b1,w2,b2,current_loss

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [37]:
    for i in range(100):
        w1, b1,w2,b2,current_loss = train(X_train, y_train, w1, b1,w2,b2)
        print("Loss at step {:d}: {:.3f}".format(i, current_loss))      #Training loss at each iteration


Loss at step 0: 273.893
Loss at step 1: 273.893
Loss at step 2: 273.893
Loss at step 3: 273.893
Loss at step 4: 273.893
Loss at step 5: 273.893
Loss at step 6: 273.893
Loss at step 7: 273.893
Loss at step 8: 273.893
Loss at step 9: 273.893
Loss at step 10: 273.893
Loss at step 11: 273.893
Loss at step 12: 273.893
Loss at step 13: 273.893
Loss at step 14: 273.893
Loss at step 15: 273.893
Loss at step 16: 273.893
Loss at step 17: 273.893
Loss at step 18: 273.893
Loss at step 19: 273.893
Loss at step 20: 273.893
Loss at step 21: 273.892
Loss at step 22: 273.892
Loss at step 23: 273.892
Loss at step 24: 273.892
Loss at step 25: 273.892
Loss at step 26: 273.892
Loss at step 27: 273.892
Loss at step 28: 273.892
Loss at step 29: 273.892
Loss at step 30: 273.892
Loss at step 31: 273.892
Loss at step 32: 273.892
Loss at step 33: 273.892
Loss at step 34: 273.892
Loss at step 35: 273.892
Loss at step 36: 273.892
Loss at step 37: 273.892
Loss at step 38: 273.892
Loss at step 39: 273.892
Loss at st

In [38]:
j =0 
for i in range(100):
        j = j+1
        w1, b1,w2,b2,current_loss = train(X_train, y_train, w1, b1,w2,b2)
        if (j==5) :
          print("Loss at step {:d}: {:.3f}".format(i, current_loss))      #Training loss at each iteration
          j=0

Loss at step 4: 273.891
Loss at step 9: 273.891
Loss at step 14: 273.891
Loss at step 19: 273.891
Loss at step 24: 273.891
Loss at step 29: 273.891
Loss at step 34: 273.891
Loss at step 39: 273.891
Loss at step 44: 273.891
Loss at step 49: 273.891
Loss at step 54: 273.891
Loss at step 59: 273.891
Loss at step 64: 273.891
Loss at step 69: 273.890
Loss at step 74: 273.890
Loss at step 79: 273.890
Loss at step 84: 273.890
Loss at step 89: 273.890
Loss at step 94: 273.890
Loss at step 99: 273.890


### Get the shapes and values of W and b

In [33]:
w1

<tf.Tensor: id=27024, shape=(4, 5), dtype=float32, numpy=
array([[ 0.56307197,  0.5974973 , -2.1882133 , -1.1882637 , -0.31819642],
       [ 0.7504402 ,  0.18279804,  0.56548864,  0.7103052 ,  0.8293352 ],
       [ 1.4857831 ,  0.44716352, -0.9485869 ,  0.62377375, -0.95944476],
       [ 0.9408965 ,  0.4884783 ,  1.7927797 , -0.36954525, -0.09329645]],
      dtype=float32)>

In [34]:
b1

<tf.Tensor: id=27027, shape=(5,), dtype=float32, numpy=
array([ 0.22770372,  0.02868305,  0.5783496 , -0.21160543,  0.30046493],
      dtype=float32)>

In [35]:
w2

<tf.Tensor: id=27018, shape=(5, 1), dtype=float32, numpy=
array([[ 1.8857754 ],
       [ 0.69347465],
       [ 1.9949948 ],
       [-0.04412334],
       [ 1.142733  ]], dtype=float32)>

In [36]:
b2

<tf.Tensor: id=27021, shape=(1,), dtype=float32, numpy=array([1.7789539], dtype=float32)>

### Model Prediction on 1st Examples in Test Dataset

In [41]:
y_pred,_=prediction(X_test,w1,b1,w2,b2)
y_pred

<tf.Tensor: id=41447, shape=(300, 1), dtype=float32, numpy=
array([[5.6059194],
       [5.6078753],
       [5.616567 ],
       [5.6022005],
       [5.632062 ],
       [5.607492 ],
       [5.603563 ],
       [5.6115804],
       [5.6055694],
       [5.6105304],
       [5.6089115],
       [5.613599 ],
       [5.6118126],
       [5.611454 ],
       [5.6085887],
       [5.6100054],
       [5.6247225],
       [5.607809 ],
       [5.6064663],
       [5.618067 ],
       [5.610692 ],
       [5.612874 ],
       [5.6042852],
       [5.6040716],
       [5.6032853],
       [5.6084433],
       [5.631293 ],
       [5.618208 ],
       [5.6409254],
       [5.6383085],
       [5.605815 ],
       [5.6176395],
       [5.6143527],
       [5.609762 ],
       [5.6254287],
       [5.611258 ],
       [5.613589 ],
       [5.6047726],
       [5.6164684],
       [5.619207 ],
       [5.6139736],
       [5.6036253],
       [5.6130733],
       [5.619395 ],
       [5.6022124],
       [5.6049895],
       [5.6216946],


In [42]:
y_test - y_pred

<tf.Tensor: id=41450, shape=(300, 300), dtype=float32, numpy=
array([[-0.7065191 , -4.7830195 , -4.3777194 , ..., -2.2190194 ,
        -3.9351194 , -1.5943193 ],
       [-0.7084751 , -4.7849755 , -4.3796754 , ..., -2.2209754 ,
        -3.9370754 , -1.5962753 ],
       [-0.7171669 , -4.7936673 , -4.388367  , ..., -2.2296672 ,
        -3.9457672 , -1.6049671 ],
       ...,
       [-0.7220769 , -4.7985773 , -4.393277  , ..., -2.2345772 ,
        -3.9506772 , -1.6098771 ],
       [-0.7078433 , -4.7843437 , -4.3790436 , ..., -2.2203436 ,
        -3.9364436 , -1.5956435 ],
       [-0.71884394, -4.7953444 , -4.390044  , ..., -2.2313442 ,
        -3.9474442 , -1.6066442 ]], dtype=float32)>

In [0]:
from sklearn import metrics

In [44]:
cm=metrics.r2_score(y_test,y_pred)
print(cm)

-0.016481337564543397


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv('/content/drive/My Drive/Internal Lab-6/11_Iris.csv')

In [7]:
iris.shape

(150, 6)

In [8]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
iris = pd.get_dummies(iris, columns=["Species"])

In [10]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
X = iris.drop(["Id", "Species_Iris-setosa", "Species_Iris-versicolor", "Species_Iris-virginica"], axis=1)

In [0]:
y = iris[["Species_Iris-setosa", "Species_Iris-versicolor", "Species_Iris-virginica"]]

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
#tf.compat.v1.disable_eager_execution()

In [14]:
from keras.models import Sequential
from keras.layers import Dense

# define base model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(4, input_dim=4, kernel_initializer='normal', activation='softmax')) # first value 13 - number of neurons in first hidden layer can be anyhting - 10, 8 
model.add(tf.keras.layers.Dense(3, kernel_initializer='normal')) #last dense id treated as output layer # regression always one output , but for classification for binary we can have onr ouput but for multiclass we would have 2-3 outputs due to one hot encoding  
# Compile model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
  

Using TensorFlow backend.


AttributeError: ignored

### Model Training 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.30,random_state = 7)

In [71]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)
          

W1201 11:50:24.416214 139859524106112 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1201 11:50:24.482742 139859524106112 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1201 11:50:24.491938 139859524106112 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1201 11:50:24.493869 139859524106112 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecate

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 0s 3ms/step - loss: 9.5550 - acc: 0.3714 - val_loss: 11.8199 - val_acc: 0.2667
Epoch 2/50
105/105 [==============================] - 0s 85us/step - loss: 10.2849 - acc: 0.3619 - val_loss: 11.8199 - val_acc: 0.2667
Epoch 3/50
105/105 [==============================] - 0s 75us/step - loss: 10.2849 - acc: 0.3619 - val_loss: 11.8199 - val_acc: 0.2667
Epoch 4/50
105/105 [==============================] - 0s 82us/step - loss: 10.2849 - acc: 0.3619 - val_loss: 11.8199 - val_acc: 0.2667
Epoch 5/50
105/105 [==============================] - 0s 75us/step - loss: 10.2849 - acc: 0.3619 - val_loss: 11.8199 - val_acc: 0.2667
Epoch 6/50
105/105 [==============================] - 0s 92us/step - loss: 10.2849 - acc: 0.3619 - val_loss: 11.8199 - val_acc: 0.2667
Epoch 7/50
105/105 [==============================] - 0s 85us/step - loss: 10.2849 - acc: 0.3619 - val_loss: 11.8199 - val_acc: 0.2667
Epoch 8/50
1

### Model Prediction

### Save the Model

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?